# Embeddings in protein language models

In [ ]:
# Uncomment & execute once to download data
#!mkdir -p data
#!curl https://alphafold.ebi.ac.uk/files/AF-P35557-F1-model_v4.pdb > data/AF-P35557-F1-model_v4.pdb

In [ ]:
import itertools, numpy as np, pandas as pd, sklearn as sk, sklearn.preprocessing, sklearn.metrics, sklearn.naive_bayes
import matplotlib, matplotlib.colors, matplotlib.pyplot as plt, seaborn as sns
import torch, datasets, evaluate, transformers # Hugging Face libraries https://doi.org/10.18653/v1/2020.emnlp-demos.6
import Bio.SeqIO.FastaIO # Biopython for reading fasta files
import py3Dmol # Visualising 3D structures; install with pip install py3Dmol

In [ ]:
# Load the smallest version of the ESM-2 language model
model_checkpoint = 'facebook/esm2_t6_8M_UR50D'
unmasker = transformers.pipeline(model=model_checkpoint)

In [ ]:
# Sequence for GCK glucokinase (P35557), see Figure 3F in https://www.science.org/doi/10.1126/science.adg7492#sec-5
peptides = 'MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLRLETHEEASVKMLPTYVRSTPEGSEVGDFLSLDLGGTNFRVMLVKVGEGEEGQWSVKTKHQMYSIPEDAMTGTAEMLFDYISECISDFLDKHQMKHKKLPLGFTFSFPVRHEDIDKGILLNWTKGFKASGAEGNNVVGLLRDAIKRRGDFEMDVVAMVNDTVATMISCYYEDHQCEVGMIVGTGCNACYMEEMQNVELVEGDEGRMCVNTEWGAFGDSGELDEFLLEYDRLVDESSANPGQQLYEKLIGGKYMGELVRLVLLRLVDENLLFHGEASEQLRTRGAFETRFVSQVESDTGDRKQIYNILSTLGLRPSTTDCDIVRRACESVSTRAAHMCSAGLAGVINRMRESRSEDVMRITVGVDGSVYKLHPSFKERFHASVRRLTPSCEITFIESEEGSGRGAALVSAVACKKACMLGQ'
print(len(peptides), 'residues in sequence')

In [ ]:
# Mask residue at the first position
pos = 0
print(peptides[pos])
peptides_masked = peptides[:pos] + '<mask>' + peptides[pos + 1:]
peptides_masked

In [ ]:
# Q1. Calculate ESM-2 scores around residue D205 of the GCK glucokinase
# Apply the ESM-2 model to the masked sequence
unmasker(peptides_masked)

In [ ]:
# Define residues with Alpha helix / Beta sheet / within a small-molecule binding site (inferred from structure using pymol)
resid_alpha_helix = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 50, 51, 52, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 238, 239, 240, 257, 258, 259, 267, 268, 269, 272, 273, 274, 275, 276, 277, 278, 279, 280, 288, 289, 290, 291, 292, 293, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 322, 323, 324, 325, 332, 333, 334, 335, 336, 337, 338, 339, 340, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 411, 412, 413, 414, 415, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464]
resid_beta_sheet = [72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 145, 146, 147, 148, 149, 150, 154, 155, 156, 157, 158, 161, 162, 163, 164, 201, 202, 203, 222, 223, 224, 225, 226, 230, 231, 232, 233, 234, 235, 236, 237, 250, 251, 252, 253, 254, 402, 403, 404, 405, 406, 407, 408, 409, 434, 435, 436, 437, 438, 439, 440]
resid_binding_sites = [61, 62, 63, 64, 65, 66, 151, 152, 153, 159, 168, 169, 204, 205, 206, 210, 211, 214, 218, 220, 221, 225, 229, 230, 231, 235, 250, 254, 256, 258, 287, 290, 451, 452, 455, 456, 459]
print(len(resid_alpha_helix), 'residues in alpha helix')
print(len(resid_beta_sheet), 'residues in as beta sheets')
print(len(resid_binding_sites), 'residues in binding sites')

In [ ]:
# Q2. Visualise alpha helices, beta sheets and binding sites
view = py3Dmol.view()
view.addModel(open('data/AF-P35557-F1-model_v4.pdb', 'r').read(), 'pdb')
view.setStyle({
    'cartoon': {
        'colorscheme': {
            'prop': 'resi',
            'map': { resi + 1: 'red' if resi + 1 in resid_alpha_helix else 'blue' for resi in range(len(peptides)) },
        },
        'arrows': True,
    }
})
view.zoomTo()
view.show()

In [ ]:
# Run the ESM-2 model on the sequence; capture full output and internal state
tokens = unmasker.tokenizer(peptides, return_tensors='pt')
outputs = unmasker.model(tokens['input_ids'], attention_mask=tokens['attention_mask'], output_hidden_states=True,)

In [ ]:
# Q3. Find the number of layers & embeddings in the hiddens states
outputs.hidden_states[0].shape

In [ ]:
# Visualise a specific embedding on the protein structure
def get_embeddings(layer_id, embedding_id):
    raw_values = outputs.hidden_states[layer_id][0,:,embedding_id][1:-1].detach().numpy()
    return sklearn.preprocessing.MinMaxScaler().fit_transform(raw_values.reshape(-1, 1)).flatten()

def get_colormap(values):
    cmap = sns.color_palette('viridis', as_cmap=True)
    pos_col = {i + 1: matplotlib.colors.to_hex(cmap(val)) for i, val in enumerate(values) }
    return pos_col

def n_layers():
    return len(outputs.hidden_states)

def n_embeddings():
    return outputs.hidden_states[-1].shape[-1]

view = py3Dmol.view()
view.addModel(open('data/AF-P35557-F1-model_v4.pdb', 'r').read(), 'pdb')
view.setStyle({
    'cartoon': {
        'colorscheme': {
            'prop': 'resi',
            'map': get_colormap(get_embeddings(layer_id=1, embedding_id=5)),
        },
        'arrows': True,
    }
})
view.zoomTo()
view.show()

In [ ]:
# Q4. Visualise "embedding probe" AUCs for alpha-helix, beta-sheet and binding site predictions across different layers
# 
y_true = [ resi + 1 in resid_alpha_helix for resi in range(len(peptides)) ]
#y_true = [ resi + 1 in resid_beta_sheet for resi in range(len(peptides)) ]
#y_true = [ resi + 1 in resid_binding_sites for resi in range(len(peptides)) ]

for layer_id in range(n_layers()):
    X = outputs.hidden_states[layer_id][0,1:-1,:].detach().numpy()
    clf = sklearn.naive_bayes.GaussianNB().fit(X, y_true)
    print(layer_id, sk.metrics.roc_auc_score(y_true=y_true, y_score=clf.predict_proba(X)[:,1]))

In [ ]:
# Q5. Visualise the performance of the top-performing layer
layer_id = 2
X = outputs.hidden_states[layer_id][0,1:-1,:].detach().numpy()
clf = sklearn.naive_bayes.GaussianNB().fit(X, y_true)
predictions = clf.predict_proba(X)[:,1]

view = py3Dmol.view()
view.addModel(open('data/AF-P35557-F1-model_v4.pdb', 'r').read(), 'pdb')
view.setStyle({
    'cartoon': {
        'colorscheme': {
            'prop': 'resi',
            'map': get_colormap(predictions),
        },
        'arrows': True,
    }
})
view.zoomTo()
view.show()